In [1]:
import pandas as pd
import numpy as np
import datetime
import json
from datetime import datetime as dt

In [2]:
import urllib.request

# Quary as defined by ministary of healthe website on 20200307, partial geographical area.
# See documentation https://developers.arcgis.com/javascript/3/jsapi/spatialreference-amd.html#wkid
# See https://gis.health.gov.il/arcgis/rest/services/CoronaExposureSites/MapServer/0
# CORONA_DATA_URL = "https://gis.health.gov.il/arcgis/rest/services/CoronaExposureSites/MapServer/0/query?f=json&where=(ID <> 'K_1') AND (OBJECTID <> 173)&returnGeometry=true&spatialRel=esriSpatialRelIntersects&maxAllowableOffset=305&geometry={"xmin":3205455.045000008,"ymin":2950407.3900000037,"xmax":3563929.7500000055,"ymax":3486931.273000005,"spatialReference":{"wkid":102100,"latestWkid":3857}}&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100"
# Quary as defined by ministary of healthe website on 20200307, with removal of all geographical limitations, and taking degrees intead of Projected Coordinate Systems. 
# CORONA_DATA_URL = "https://gis.health.gov.il/arcgis/rest/services/CoronaExposureSites/MapServer/0/query?f=json&where=(ID <> 'K_1') AND (OBJECTID <> 173)&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&outFields=*"
# removed all query limitations
# CORONA_DATA_URL = "https://gis.health.gov.il/arcgis/rest/services/CoronaExposureSites/MapServer/0/query?f=json&where=(ID <> 'XXXX')&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&outFields=*"
# encoded
CORONA_DATA_URL = "https://gis.health.gov.il/arcgis/rest/services/CoronaExposureSites/MapServer/0/query?f=json&where=(ID%20%3C%3E%20%27XXXX%27)&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometryType=esriGeometryEnvelope&outFields=*"
with urllib.request.urlopen(CORONA_DATA_URL) as response:
    corona_exposures_sites_query_data = json.loads(response.read().decode('utf-8'))

In [3]:
corona_exposures_features = pd.json_normalize(corona_exposures_sites_query_data['features'])
corona_exposures_features

def convertTimestampMsToDatetime(df, input_column, output_column):
    df[output_column] = (df[input_column].astype(float) / 1000).map(lambda x: None if pd.isna(x) else dt.fromtimestamp(x))
    

def convertTypesStrToLiteral(df):
    import ast
    df['attributes.types'] = df['attributes.types'].map(lambda x: ast.literal_eval(x) if x and str(x).startswith("[") else x)

convertTimestampMsToDatetime(corona_exposures_features, 'attributes.fromTime', 'attributes.fromDatetime')
convertTimestampMsToDatetime(corona_exposures_features, 'attributes.toTime', 'attributes.toDatetime')
convertTimestampMsToDatetime(corona_exposures_features, 'attributes.flightArrival', 'attributes.flightArrivalDatetime')
convertTypesStrToLiteral(corona_exposures_features)
corona_exposures_features = corona_exposures_features.drop(labels=['attributes.Date', 'attributes.POINT_X', 'attributes.POINT_Y', 'attributes.fromTime', 'attributes.toTime', 'attributes.flightArrival'], axis=1, inplace=False)
# remove korean turiests, they are removed in the original query and don't have exact times, and long ago
corona_exposures_features = corona_exposures_features[corona_exposures_features['attributes.ID'] != 'K_1']
corona_exposures_no_flight = corona_exposures_features
corona_exposures_no_flight = corona_exposures_no_flight.drop(labels=['attributes.flight', 'attributes.flightFrom', 'attributes.flightArrivalDatetime'], axis=1, inplace=False)
corona_exposures_flights = corona_exposures_features[['attributes.flight', 'attributes.flightFrom', 'attributes.flightArrivalDatetime']]
corona_exposures_flights = corona_exposures_flights[corona_exposures_flights['attributes.flight'].map(bool) & (corona_exposures_flights['attributes.flight'] != 'לא רלוונטי')]
corona_exposures_flights = corona_exposures_flights.drop_duplicates(subset=['attributes.flight','attributes.flightFrom'])